In [30]:
# %reset
from sympy import *
import numpy as np 
init_printing()

comp, oop = symbols( 'comp, oop' )

# Definitions
eta, dx, dz = symbols( 'eta, dx, dz' )
D11, D12, D13, D14, D21, D22, D23, D24 ,D31, D32, D33, D34, D41, D42, D43, D44 = symbols('D11, D12, D13, D14, D21, D22, D23, D24 ,D31, D32, D33, D34, D41, D42, D43, D44')
Exx, Ezz, Gxz, divV = symbols('Exx, Ezz, Gxz, divV')
stress_labels = ['Txx', 'Tzz', 'Txz', 'P']

# Isotropic
Dv = Matrix( [ [D11, 0, 0, 0], [0, D22, 0, 0], [0, 0, D33, 0], [0, 0, 0, 0]  ] )

# Anisotropic
Dv = Matrix( [ [D11, D12, D13, D14], [D21, D22, D23, D24], [D31, D32, D33, D34], [D41, D42, D43, D44]  ] )

# Check constitutive
Ed = Matrix( [ [Exx], [Ezz], [Gxz], [divV] ] )
T  = Dv*Ed
for i in range(4):
    print(stress_labels[i] + ' = ' + ccode( T[i]) + ';' )

Txx = D11*Exx + D12*Ezz + D13*Gxz + D14*divV;
Tzz = D21*Exx + D22*Ezz + D23*Gxz + D24*divV;
Txz = D31*Exx + D32*Ezz + D33*Gxz + D34*divV;
P = D41*Exx + D42*Ezz + D43*Gxz + D44*divV;


In [32]:
# Compressibility switch
comp = symbols('comp')

# Stencil x
uW,uC,uE,uS,uN,vSW,vSE,vNW,vNE = symbols('uW,uC,uE,uS,uN,vSW,vSE,vNW,vNE')
dofs = Matrix([uW,uC,uE,uS,uN,vSW,vSE,vNW,vNE])

# Stencil extension 1
uSW,uSE,uNW,uNE,vSWW,vSEE,vNWW,vNEE,vSSW,vSSE,vNNW,vNNE = symbols('uSW,uSE,uNW,uNE,vSWW,vSEE,vNWW,vNEE,vSSW,vSSE,vNNW,vNNE')
dofs_ext = Matrix([uSW,uSE,uNW,uNE,vSWW,vSEE,vNWW,vNEE,vSSW,vSSE,vNNW,vNNE])
dofs = np.append(dofs,dofs_ext)

# Stencil extension 2
pW, pE, pSW, pSE, pNW, pNE = symbols('pW, pE, pSW, pSE, pNW, pNE')
dofs_ext                   = Matrix([pW, pE, pSW, pSE, pNW, pNE])
dofs                       = np.append(dofs,dofs_ext)

# Flags
inS,inN,inW,inE         = symbols('inS,inN,inW,inE')
inSEc,inSWc,inNEc,inNWc = symbols('inSEc,inSWc,inNEc,inNWc')
inSEv,inSWv,inNEv,inNWv = symbols('inSEv,inSWv,inNEv,inNWv')

# Divergences
divW      = oop*( (uC  - uW )/dx + (vNW - vSW )/dz ) 
divE      = oop*( (uE  - uC )/dx + (vNE - vSE )/dz )
divSW     = oop*( (uS  - uSW)/dx + (vSW - vSSW)/dz )
divSE     = oop*( (uSE - uS )/dx + (vSE - vSSE)/dz )
divNW     = oop*( (uN  - uNW)/dx + (vNNW- vNW )/dz )
divNE     = oop*( (uNE - uN )/dx + (vNNE- vNE )/dz )

# Deviatoric normal strain rate
ExxW      = ((uC -uW )/dx - comp*1/3*divW) 
EzzW      = ((vNW-vSW)/dz - comp*1/3*divW)
ExxE      = ((uE -uC )/dx - comp*1/3*divE)
EzzE      = ((vNE-vSE)/dz - comp*1/3*divE)

# Additional missing stress tensor components needed for interpolation
GxzNE     = ((uNE - uE )/dz + ( vNEE - vNE  )/dx)
GxzSE     = ((uE  - uSE)/dz + ( vSEE - vSE  )/dx)
GxzNW     = ((uNW - uW )/dz + ( vNW  - vNWW )/dx)
GxzSW     = ((uW  - uSW)/dz + ( vSW  - vSWW )/dx)

# Shear strain rate
GxzN      = ((uN - uC )/dz + (vNE -vNW )/dx)         # clear
GxzS      = ((uC - uS )/dz + (vSE -vSW )/dx) 

ExxNE     = ((uNE -uN )/dx - comp*1/3*divNE)
ExxNW     = ((uN  -uNW)/dx - comp*1/3*divNW)
ExxSE     = ((uSE -uS )/dx - comp*1/3*divSE)
ExxSW     = ((uS  -uSW)/dx - comp*1/3*divSW)
EzzNE     = ((vNNE-vNE)/dz - comp*1/3*divNE)
EzzNW     = ((vNNW-vNW)/dz - comp*1/3*divNW)
EzzSE     = ((vSE-vSSE)/dz - comp*1/3*divSE)
EzzSW     = ((vSW-vSSW)/dz - comp*1/3*divSW)
# INV 0
GxzE      = 0.25*( inN*GxzN + inS*GxzS + inNEv*GxzNE + inSEv*GxzSE)      # clear
GxzW      = 0.25*( inN*GxzN + inS*GxzS + inNWv*GxzNW + inSWv*GxzSW)
ExxN      = 0.25*( inE*ExxE + inW*ExxW + inNWc*ExxNW + inNEc*ExxNE)
ExxS      = 0.25*( inE*ExxE + inW*ExxW + inSWc*ExxSW + inSEc*ExxSE)
EzzN      = 0.25*( inE*EzzE + inW*EzzW + inNWc*EzzNW + inNEc*EzzNE) 
EzzS      = 0.25*( inE*EzzE + inW*EzzW + inSWc*EzzSW + inSEc*EzzSE)
pS        = 0.25*( inE*pE   + inW*pW   + pSW   + pSE)
pN        = 0.25*( inE*pE   + inW*pW   + pNW   + pNE)  

#----------------------------------------#
EdW  = Matrix([[ExxW], [EzzW], [GxzW], [pW]])
TW   = Dv*EdW
TxxW = TW[0].subs({D11:'D11W', D12:'D12W', D13:'D13W', D14:'D14W'})
#----------------------------------------#
EdE  = Matrix([[ExxE], [EzzE], [GxzE], [pE]])
TE   = Dv*EdE
TxxE = TE[0].subs({D11:'D11E', D12:'D12E', D13:'D13E', D14:'D14E'})
#----------------------------------------#
EdS  = Matrix([[ExxS], [EzzS], [GxzS], [pS]])
TS   = Dv*EdS
TxzS = TS[2].subs({D31:'D31S', D32:'D32S', D33:'D33S', D34:'D34S'})
#----------------------------------------#
EdN  = Matrix([[ExxN], [EzzN], [GxzN], [pN]])
TN   = Dv*EdN
TxzN = TN[2].subs({D31:'D31N', D32:'D32N', D33:'D33N', D34:'D34N'})
#----------------------------------------#
Fx = 1/dx*(inE*TxxE - inW*TxxW) + 1/dz*(inN*TxzN - inS*TxzS) - 1/dx*(inE*pE - inW*pW)
Fx = -Fx
for i in range(len(dofs)):
    cUc = Fx.diff(dofs[i])
    print(str(dofs[i]) + ' = ' +  ccode(simplify(cUc)) + ';')

uW = (1.0/3.0)*inW*(-0.75*D13W*dx*(inNWv - inSWv) + 0.25*dx*(-inN*(D31N*(comp*oop - 3) + D32N*comp*oop) + inS*(D31S*(comp*oop - 3) + D32S*comp*oop)) + dz*(D11W*(comp*oop - 3) + D12W*comp*oop))/(pow(dx, 2)*dz);
uC = (dx*(-inN*(-D33N*dx + dz*(inE - inW)*(D31N*(0.083333333333333329*comp*oop - 0.25) + 0.083333333333333329*D32N*comp*oop)) + inS*(D33S*dx + dz*(inE - inW)*(D31S*(0.083333333333333329*comp*oop - 0.25) + 0.083333333333333329*D32S*comp*oop))) - 1.0/3.0*dz*(inE*(-0.75*D13E*dx*(inN - inS) + dz*(D11E*(comp*oop - 3) + D12E*comp*oop)) + inW*(0.75*D13W*dx*(inN - inS) + dz*(D11W*(comp*oop - 3) + D12W*comp*oop))))/(pow(dx, 2)*pow(dz, 2));
uE = (1.0/3.0)*inE*(0.75*D13E*dx*(inNEv - inSEv) + 0.25*dx*(inN*(D31N*(comp*oop - 3) + D32N*comp*oop) - inS*(D31S*(comp*oop - 3) + D32S*comp*oop)) + dz*(D11E*(comp*oop - 3) + D12E*comp*oop))/(pow(dx, 2)*dz);
uS = inS*(-D33S*dx + dz*(inSEc - inSWc)*(D31S*(0.083333333333333329*comp*oop - 0.25) + 0.083333333333333329*D32S*comp*oop) + 0.25*dz*(D13E*inE - D1

In [31]:
# Compressibility switch
comp = symbols('comp')

# Stencil z
vW,vC,vE,vS,vN,uSW,uSE,uNW,uNE = symbols('vW,vC,vE,vS,vN,uSW,uSE,uNW,uNE')
dofs = Matrix([vW,vC,vE,vS,vN,uSW,uSE,uNW,uNE])

# Stencil extension 1
vSW,vSE,vNW,vNE,uSWW,uSEE,uNWW,uNEE,uSSW,uSSE,uNNW,uNNE = symbols('vSW,vSE,vNW,vNE,uSWW,uSEE,uNWW,uNEE,uSSW,uSSE,uNNW,uNNE')
dofs_ext = Matrix([vSW,vSE,vNW,vNE,uSWW,uSEE,uNWW,uNEE,uSSW,uSSE,uNNW,uNNE])
dofs = np.append(dofs,dofs_ext)

# Stencil extension 2
pS, pN, pSW, pSE, pNW, pNE = symbols('pS, pN, pSW, pSE, pNW, pNE')
dofs_ext                   = Matrix([pS, pN, pSW, pSE, pNW, pNE])
dofs                       = np.append(dofs,dofs_ext)

# Flags
inS,inN,inW,inE         = symbols('inS,inN,inW,inE')
inSEc,inSWc,inNEc,inNWc = symbols('inSEc,inSWc,inNEc,inNWc')
inSEv,inSWv,inNEv,inNWv = symbols('inSEv,inSWv,inNEv,inNWv')

# Divergences
divS      = oop*( (uSE-uSW)/dx + (vC -vS )/dz )   
divN      = oop*( (uNE-uNW)/dx + (vN -vC )/dz )  
divSW     = oop*( (uSW-uSWW)/dx + (vW -vSW )/dz )
divSE     = oop*( (uSEE-uSE)/dx + (vE -vSE )/dz )
divNW     = oop*( (uNW-uNWW)/dx + (vNW -vW )/dz )
divNE     = oop*( (uNEE-uNE)/dx + (vNE -vE )/dz ) 

# Deviatoric normal strain
ExxS      = ((uSE-uSW)/dx - comp*1/3*divS)
EzzS      = ((vC -vS )/dz - comp*1/3*divS)
ExxN      = ((uNE-uNW)/dx - comp*1/3*divN)
EzzN      = ((vN -vC )/dz - comp*1/3*divN)

# Additional missing stress tensor components via interpolation
GxzSW     = ((uSW  - uSSW)/dz + (vS  - vSW)/dx)
GxzSE     = ((uSE  - uSSE)/dz + (vSE - vS )/dx)
GxzNW     = ((uNNW - uNW )/dz + (vN  - vNW)/dx)
GxzNE     = ((uNNE - uNE )/dz + (vNE - vN )/dx)

# Shear strain rate
GxzE      = (uNE-uSE)/dz + (vE - vC )/dx  # clear
GxzW      = (uNW-uSW)/dz + (vC - vW )/dx

# Additional missing stress tensor components for interpolation
ExxNE     = ((uNEE-uNE)/dx - comp*1/3*(divNE))
ExxNW     = ((uNW-uNWW)/dx - comp*1/3*(divNW))
ExxSE     = ((uSEE-uSE)/dx - comp*1/3*(divSE))
ExxSW     = ((uSW-uSWW)/dx - comp*1/3*(divSW))
EzzNE     = ((vNE -vE )/dz - comp*1/3*(divNE))
EzzNW     = ((vNW -vW )/dz - comp*1/3*(divNW))
EzzSE     = ((vE -vSE )/dz - comp*1/3*(divSE))
EzzSW     = ((vW -vSW )/dz - comp*1/3*(divSW))

# INV 0
GxzN      = 0.25*( inW*GxzW + inE*GxzE + inNWv*GxzNW + inNEv*GxzNE )
GxzS      = 0.25*( inW*GxzW + inE*GxzE + inSWv*GxzSW + inSEv*GxzSE ) 
ExxE      = 0.25*( inS*ExxS + inN*ExxN + inNEc*ExxNE + inSEc*ExxSE )
ExxW      = 0.25*( inS*ExxS + inN*ExxN + inNWc*ExxNW + inSWc*ExxSW )
EzzE      = 0.25*( inS*EzzS + inN*EzzN + inNEc*EzzNE + inSEc*EzzSE )
EzzW      = 0.25*( inS*EzzS + inN*EzzN + inNWc*EzzNW + inSWc*EzzSW )
pE        = 0.25*( inS*pS   + inN*pN   + inNEc*pNE   + inSEc*pSE )
pW        = 0.25*( inS*pS   + inN*pN   + inNWc*pNW   + inSWc*pSW )

#----------------------------------------#
EdS  = Matrix([[ExxS], [EzzS], [GxzS], [pS]])
TS   = Dv*EdS
TzzS = TS[1].subs({D21:'D21S', D22:'D22S', D23:'D23S', D24:'D24S'})
#----------------------------------------#
EdN  = Matrix([[ExxN], [EzzN], [GxzN], [pN]])
TN   = Dv*EdN
TzzN = TN[1].subs({D21:'D21N', D22:'D22N', D23:'D23N', D24:'D24N'})
#----------------------------------------#
EdW  = Matrix([[ExxW], [EzzW], [GxzW], [pW]])
TW   = Dv*EdW
TxzW = TW[2].subs({D31:'D31W', D32:'D32W', D33:'D33W', D34:'D34W'})
#----------------------------------------#
EdE  = Matrix([[ExxE], [EzzE], [GxzE], [pE]])
TE   = Dv*EdE
TxzE = TE[2].subs({D31:'D31E', D32:'D32E', D33:'D33E', D34:'D34E'})
#----------------------------------------#
Fz = 1/dz*(inN*TzzN - inS*TzzS) + 1/dx*(inE*TxzE - inW*TxzW) - (inN*pN - inS*pS)/dz 
Fz = -Fz
for i in range(len(dofs)):
    cVc = Fz.diff(dofs[i])
    print(str(dofs[i]) + ' = ' +  ccode(simplify(cVc)) + ';') 

vW = inW*(-D33W*dz + dx*(inNWc - inSWc)*(0.083333333333333329*D31W*comp*oop + D32W*(0.083333333333333329*comp*oop - 0.25)) + 0.25*dx*(D23N*inN - D23S*inS))/(pow(dx, 2)*dz);
vC = (-1.0/3.0*dx*(inN*(-0.75*D23N*dz*(inE - inW) + dx*(D21N*comp*oop + D22N*(comp*oop - 3))) + inS*(0.75*D23S*dz*(inE - inW) + dx*(D21S*comp*oop + D22S*(comp*oop - 3)))) + dz*(-inE*(-D33E*dz + dx*(inN - inS)*(0.083333333333333329*D31E*comp*oop + D32E*(0.083333333333333329*comp*oop - 0.25))) + inW*(D33W*dz + dx*(inN - inS)*(0.083333333333333329*D31W*comp*oop + D32W*(0.083333333333333329*comp*oop - 0.25)))))/(pow(dx, 2)*pow(dz, 2));
vE = inE*(-D33E*dz - dx*(inNEc - inSEc)*(0.083333333333333329*D31E*comp*oop + D32E*(0.083333333333333329*comp*oop - 0.25)) + 0.25*dx*(-D23N*inN + D23S*inS))/(pow(dx, 2)*dz);
vS = (1.0/3.0)*inS*(-0.75*D23S*dz*(inSEv - inSWv) + dx*(D21S*comp*oop + D22S*(comp*oop - 3)) + 0.25*dz*(-inE*(D31E*comp*oop + D32E*(comp*oop - 3)) + inW*(D31W*comp*oop + D32W*(comp*oop - 3))))/(dx*pow(dz, 2));
vN = (1